### Installing the prerequisites for using the Pinecone Vector Database

In [ ]:


!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
# this tiktoken helps us to call the openAI embedding
!pip install tiktoken
pip install -U langchain-community

In [ ]:
pip install sentence-transformers

In [ ]:
pip install transformers


In [ ]:
# Huggingface hub has various repositories by which we call various opensource llm models
!pip install langchain-huggingface 
## For API Calls
!pip install huggingface_hub  

### Generating API Key in pinecone for using the vectordatabase of pinecone

Open the Pinecone webiste 

https://www.google.com/search?q=pincone.

- Get sign up 
- since we are using the free tier account of pinecone so we can able to create only 1 Index which helps us to generate the pinecone API key
- click on API key where we can see the default API key and generated API by creation of index 
- By using any one API key bydefault or generated API key by creation if index we can able to use the vector database of pinecone

- copy any one of the API key of pinecone in API key section 



### Importing the necessary modules from langchain because langchain framework acts as wrapper in OPenAI ,pinecone,huggingface etc langchain consist of everything which are required  to develop the generative AI Application 

In [1]:

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

### Lets create the Folder in the workspace where we can storing the data which we furthur convert the data into vector embedding and store in pinecone vector database and furthur we pass it to llm model 

In [2]:
mkdir pdfs

A subdirectory or file pdfs already exists.


In [3]:
loader=PyPDFDirectoryLoader('pdfs')  # by using PyPDFDirectoryLoader i could able to load the transformer research paper which is present in pdf folder and saved the attention is all you need research pdf data in loader variable

In [4]:
data=loader.load()
print(data)  # as we can see the output we successfully loaded the pdf data and saved in data variable

[Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring

### Performing the Tokenization on data by using RecursiveCharacterTextSplitter()

RecursiveCharacterTextSplitter helps us to split the pdf data content in the form of Chunks  , here chunks stands for small paragraph which has meaningful context so like this way we can divide the whole paragraph chunk into mentioned desired no of chunks (paragraphs) by using this RecursiveCharacterTextSplitter 

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)  # By using RecursiveCharacterTextSplitter class and passing the 2 arguments iam able to perform the tokenization and i have created the object for this class

In [6]:
text_chunks = text_splitter.split_documents(data)  # with the help of the created object for the class iam calling one of the class method which is split_document which perform 

In [7]:
text_chunks

[Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or', metadata={'source': 'pdfs\\attention-is-all-you-need-Research Paper.pdf', 'page': 0}),
 Document(page_content='convolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two mach

In [8]:
len(text_chunks)  # the total chunks or smaller paragraphs which our methid split_documents converted into 76 chunks or smaller documents or paragraph

76

In [9]:
print(text_chunks[0].page_content)   # acessing the each chunk or document or paragraph by index value of list pf text_chunks

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or


In [10]:
print(text_chunks[2].page_content) 

less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.0 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature.
1 Introduction


In [11]:
print(text_chunks[75].page_content)  # so like this way we can access the max index of 75 which is our last chunk or last small paragraph of the pdf data 

[31] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang
Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation. arXiv preprint
arXiv:1609.08144 , 2016.
[32] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR , abs/1606.04199, 2016.
11


## Fetching the Pre trained vector embedded transformer from Huggingface application  (Encoding Techinque with Pretrained model)
- Setting up the Huggingface API KEY for using the Pretrained vector Embedding Model for converting the text data into embedding dense vectors or continues vector space  to capture the semantic meaning between the each feature the pretrained trained on vs each input text data which we have in our pdf 


In [12]:
from dotenv import load_dotenv  # Python-dotenv reads key-value pairs from a .env file and can set them as environment variables., we use this library to read the open AI api key secretly rather than exposing it to public it helps us to read the key in secrete and than after mentioning the key value pair of openai key in .env file makesure add this .env file to .gitiognore so it wont get push to github

load_dotenv()  # take environment variables from .env.

True

In [13]:
# reading the hugginface api key token from .env file 
KEY1=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
# Load the pretrained vector embedded model
from langchain_community.embeddings import SentenceTransformerEmbeddings
huggingface_embeddings_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # i loaded the pre trained embedded model here which is all-MiniLM-L6-v2


### Steps for Saving the embedded dense vector in the pinecode vectordatabase

Open the Pinecone webiste 

https://www.google.com/search?q=pincone.

- Get sign up 
- click on API key where we can see the default API key and generated API key by clicking on new option
- By using any one API key bydefault or generated API key we can able to use the vector database of pinecone to store the embedded vector

- copy any one of the API key and API KEY name in pinecone website of API key section 
- click on index section -> give name of the index i given name as test -> fill the dimension , metrics accoridng to the vector embedding model we are working on 
- In our case we are working on pre trained vector embedded model which we fetched from hugging face and the name of the model is all-MiniLM-L6-v2 which this model got trained on 384 dimensions of features and metrics it followed was cosine so give those values while creating index -> choose AWS -> click on create 

- Interacting with the Index: Once the index is initialized, you can perform operations like inserting and querying vectors.

In [15]:
# fetching the unique api key and index name of pine cone which helps us to save the embedded dense vector in this pinecone vector database
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')  # API Key value which i saved in .env for security purpose and fetching the key value name here through os.getenv, so we can prevint the pinecone api key to display in public
PINECONE_API_index_env='test' # api index name which we need to create for storing the vectors in pinecone database

### Initilizing the Pinecone vector Database connection

#### Pinecone is being running on top of cloud server so we can use pinecone server which is fully managed by AWS, Azure, GCP
- Just by creating the Index in the pinecone web application 
- Interacting with the Index: Once the index is initialized, you can perform operations like inserting and querying vectors.

In [ ]:
pip install --upgrade langchain-pinecone


### Create vector Embeddings for each of the Text Chunk by using Pre trained embedded model from hugging face

In [35]:
from langchain.vectorstores import Pinecone as PineconeStore
docsearch = PineconeStore.from_texts([t.page_content for t in text_chunks],huggingface_embeddings_model, index_name=PINECONE_API_index_env)  # here i pass the 75 small indiviual documents chunks or small paragraphs to the model for converting the each individual  document chunk into dense vector embedding

In [25]:
docsearch

In [27]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000002D216B9DC90>)

In [28]:
# Perform similarity search by passing the users query
query = "what is attention is need you all mean by"

In [29]:
docs = docsearch.similarity_search(query)

In [30]:
docs

[Document(page_content='masking, combined with fact that the output embeddings are offset by one position, ensures that the\npredictions for position ican depend only on the known outputs at positions less than i.\n3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum'),
 Document(page_content='masking, combined with fact that the output embeddings are offset by one position, ensures that the\npredictions for position ican depend only on the known outputs at positions less than i.\n3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum'),
 Document(page_content='As side beneﬁt, self-attention could yield more interpretable models. We inspect attention distributions\n

Just go to pinecone website https://app.pinecone.io/organizations/-O0ENPVkKNP5bYwVve4Y/projects/3b164a50-1f44-40dd-93c4-589c4792104c/indexes/test/browser

see the index browser 
we can check the total information of saved dense vector embeddings in the pinecone vector database 
- we can check the similarity score for the each dense vector embedding out of total 75 dense embedding vector 
- we can check the ID , espected dense vector Values 


### Performing the similarity search  by llm which takes the retrieval documents of vector embeddings which got stored in pinecone vector database 

In [31]:
from langchain import HuggingFaceHub # ascessing the llms models of openAI API by langchain framework
from langchain_huggingface import HuggingFaceEndpoint

# mistralai/Mistral-7B-Instruct-v0.2 , microsoft/Phi-3-mini-4k-instruct
# now i have created object for the HuggingfaceHUb class in that object i can able to collect the specified repo_id of model  in llm variable
llm =HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature= 0.5,
    token=KEY1
)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\mahen\.cache\huggingface\token
Login successful


In [39]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [40]:
# Perform similarity search by passing the users query
query = "what is attention is need you all mean by"

In [41]:
qa.run(query)

c:\Users\mahen\anaconda_new\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' In the context of the passage, attention refers to an attention function in a neural network, specifically in the context of a sequence-to-sequence model. An attention function maps a query and a set of key-value pairs to an output, where the output is computed as a weighted sum of the values. In this case, the attention function is used to compute the output for a given position in the sequence, based on the known outputs at positions less than that position. The "masking" mentioned in the passage refers to the fact that the model only considers the known outputs when computing the output for a given position, rather than considering all possible outputs. The "offset by one position" refers to the fact that the output embeddings for a given position are shifted by one position compared to the input embeddings, which allows the model to compute the output for a given position based on the known outputs at positions less than that position. Overall, attention is a mechanism that allow

In [43]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Answer:  In the context of the passage, attention refers to an attention function in a neural network, specifically in the context of a sequence-to-sequence model. An attention function maps a query and a set of key-value pairs to an output, where the output is computed as a weighted sum of the values. In this case, the attention function is used to compute the output for a given position in the sequence, based on the known outputs at positions less than that position. The "masking" mentioned in the passage refers to the fact that the model only considers the known outputs when computing the output for a given position, rather than considering all possible outputs. The "offset by one position" refers to the fact that the output embeddings for a given position are shifted by one position compared to the input embeddings, which allows the model to compute the output for a given position based on the known outputs at positions less than that position. Overall, attention is a mechanism tha

KeyboardInterrupt: Interrupted by user